# Q2


*   In this question we will Build a bigram HMM tagger.
*   First we split the part-of-speech-tagged corpus into a training set and test set.
*   From the labeled training set, we train the transition and observation probabilities of the HMM tagger directly on the hand-tagged data.
*   Then implement the Viterbi algorithm so we can decode a test sentence.
*   Run the algorithm on the test set. Report its error rate and
compare its performance to the most frequent tag baseline.
*   Build a confusion matrix and investigate the most frequent errors.


In [19]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [20]:
import math
import nltk
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [21]:
def collect_probabilities(_samples):
    """
    Collects various probabilities and frequencies required for Hidden Markov Model (HMM) training based on given samples.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing training data.

    Returns:
    tuple: A tuple containing four dictionaries:
           - tag_freq: A dictionary storing the frequency of each tag type.
           - word_per_tag_freq: A nested dictionary storing the frequency of each word under each tag.
           - bigram: A nested dictionary representing the bigram probabilities between tag types.
           - pi: A dictionary storing the initial probability distributions for tag types.

    Comments:
    - tag_freq: Stores the frequency of each tag type observed in the training data.
    - word_per_tag_freq: Stores the frequency of each word observed under each tag in the training data.
      - Each word under a specific tag is initialized with a count of 2 to provide initial smoothing.
    - bigram: Represents the transition probabilities between tag types based on bigram counts.
    - pi: Stores the initial probability distributions for tag types based on the second tag in each sample.
    """
    tag_freq = {}
    word_per_tag_freq = {}
    bigram = {}
    pi = {}
    samples_len = len(_samples)

    # Iterate through samples to collect tag frequencies, word frequencies under each tag, and initial probabilities
    for i in range(samples_len):
        sample = _samples[i]
        has_next = i + 1 < samples_len

        # Collect data for initial probabilities (pi)
        if has_next:
            if _samples[i + 1][1] not in pi:
                pi.update({_samples[i + 1][1]: 2})
            else:
               pi[_samples[i + 1][1]] += 1

        # Count tag frequencies (tag_freq) and word frequencies under each tag (word_per_tag_freq)
        if sample[1] not in tag_freq:
            tag_freq.update({sample[1]: 2})
            word_per_tag_freq.update({sample[1]: {sample[0]: 2}})
        else:
            tag_freq[sample[1]] += 1
            if sample[0] not in word_per_tag_freq[sample[1]]:
                word_per_tag_freq[sample[1]].update({sample[0]: 2})
            else:
                word_per_tag_freq[sample[1]][sample[0]] += 1


    # Initialize the bigram matrix with default counts
    for tag_0 in tag_freq:
        bigram.update({tag_0: {}})
        for tag_1 in tag_freq:
            bigram[tag_0].update({tag_1: 1})

    # Count bigram occurrences
    for i in range(samples_len - 1):
        if _samples[i][1] in bigram and _samples[i + 1][1] in bigram[_samples[i][1]]:
            bigram[_samples[i][1]][_samples[i + 1][1]] += 1

    # Return collected probabilities and frequencies
    return tag_freq, word_per_tag_freq, bigram, pi


In [29]:
def create_confusion_matrix(predictions, hidden_state, confusion_matrices , is_correct):
    """
    Updates the confusion matrices based on the predictions and hidden states,
    calculates the number of correct predictions, and identifies tags with
    the most false positives and false negatives.

    Args:
    predictions (list): Predicted sequence of tags for the given input data.
    hidden_state (list): Actual sequence of tags for the given input data.
    confusion_matrices (dict): A dictionary containing confusion matrices for each tag.
    is_correct (int): Counter for the number of correct predictions.

    Returns:
    int: Updated count of correct predictions after processing the input data.

    Comments:
    - As the Viterbi algorithm processes each sentence in the test data,
      it compares the predicted tags against the actual tags for each word in the sentence.
        - For each word in the sentence, if the predicted tag matches the actual tag (a true positive),
          the corresponding count in the confusion matrix for that tag is incremented.
        - If the predicted tag does not match the actual tag:
            - If the predicted tag is incorrect (a false positive), the FP count for
              the predicted tag is incremented, and the TN counts for other tags are incremented.
            - If the actual tag is not predicted (a false negative), the FN count for the actual
              tag is incremented, and the TN counts for other tags are incremented.
    - After updating the confusion matrices, the function identifies tags with
      the highest counts of false positives and false negatives.
    """
    # fill in the confusion matrix
    for i in range(len(predictions)):
        if predictions[i] == hidden_state[i]:
            # prediction is correct
            is_correct += 1
            for j in confusion_matrices:
                if j == predictions[i]:
                    confusion_matrices[j]['TP'] += 1
                else:
                    confusion_matrices[j]['TN'] += 1
        else:
            # prediction is incorrect
            for j in confusion_matrices:
                if j == predictions[i]:
                    confusion_matrices[j]['FP'] += 1
                elif j == hidden_state[i]:
                    confusion_matrices[j]['FN'] += 1
                else:
                    confusion_matrices[j]['TN'] += 1


    # rank
    highest_FP_tag = []
    highest_FP = -1
    highest_FN_tag = []
    highest_FN = -1

    for tag in confusion_matrices:

        print(f'{tag}: {confusion_matrices[tag]}')

        if confusion_matrices[tag]['FP'] > highest_FP:
            highest_FP = confusion_matrices[tag]['FP']
            highest_FP_tag = tag

        if confusion_matrices[tag]['FN'] > highest_FN:
            highest_FN = confusion_matrices[tag]['FN']
            highest_FN_tag = tag

    print(f'\nTag with the most false positives is: {highest_FP_tag} with {highest_FP} counts.')
    print(f'Tag with the most false negative is:  {highest_FN_tag} with {highest_FN} counts.')

    print(f'confusion_matrices: {confusion_matrices}')


    return is_correct


In [30]:
def viterbi(_samples, _tag_freq, _word_per_tag_freq, _bigram, _init_dist):
    """
    Performs the Viterbi algorithm for part-of-speech tagging on a given test set of sentences.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing the test data.
    _tag_freq (dict): A dictionary storing the frequency of each tag type observed in the training data.
    _word_per_tag_freq (dict): A nested dictionary storing the frequency of each word observed under each tag in the training data.
    _bigram (dict): A nested dictionary representing the bigram probabilities between tag types.
    _init_dist (dict): A dictionary storing the initial probability distributions for tag types.

    Comments:
    - This function implements the Viterbi algorithm for part-of-speech tagging.
    - It processes each sentence in the test set, using '.' as an indicator that a sentence is over.
    - The function populates a confusion matrix to track the performance of the model.
    - The Viterbi algorithm is used to find the most likely sequence of tags for each sentence.
    - The algorithm iterates through each word in the sentence, calculating the most likely tag sequence based on probabilities and transition probabilities.
    - The model's predictions are compared against the actual tags to evaluate accuracy and populate the confusion matrix.
    - The function prints detailed information about predictions, including the sentence, hidden states, and predicted tags.
    - It also identifies tags with the most false positives and false negatives based on the confusion matrix.
    - Finally, the function prints the overall accuracy of the model on the test set.
    """
    test_size = len(_samples)
    current_index = 0
    is_correct = 0

    # populate the confusion matrix
    confusion_matrices = {}
    for tag in _tag_freq:
        confusion_matrices.update({tag: {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}})

    while current_index < test_size:
        # find the boundry of a sentence
        sentence = []
        hidden_state = []
        last_token = ''
        while last_token != '.' and current_index < test_size:
            sentence.append(_samples[current_index][0])
            hidden_state.append(_samples[current_index][1])
            last_token = sentence[-1]
            current_index += 1

        # initialization step
        path_probability = {}
        backpointer = {}
        for tag in init_dist:
            path_probability.update({tag: []})
            backpointer.update({tag: [0]})

            # initial distribution of this tag
            pi_tag = init_dist[tag]

            # b_word is the probability of the word being generated by this tag
            if sentence[0] in _word_per_tag_freq[tag]:
                b_word = _word_per_tag_freq[tag][sentence[0]]
            else:
                b_word = 2.2250738585072014e-100
            path_probability[tag].append(math.log(pi_tag * b_word, 10))

        # recursion step
        T = len(sentence)
        for i in range(1, T):
            for tag in init_dist:

                if sentence[i] in _word_per_tag_freq[tag]:
                    b_word = _word_per_tag_freq[tag][sentence[i]]
                else:
                    b_word = 2.2250738585072014e-100

                # search the maximum value of
                # viterbi[s', t-1] * a(s|s') *b_s(o_t)
                best_trans_prob = -2.2250738585072014e+308
                best_trans_tag = ''
                for prev_tag in init_dist:
                    if prev_tag in _bigram and tag in _bigram[prev_tag]:
                        transitional_prob = _bigram[prev_tag][tag]
                    else:
                        transitional_prob = 2.2250738585072014e-100

                    prob = path_probability[prev_tag][i - 1] + math.log(transitional_prob * b_word, 10)

                    if prob > best_trans_prob:
                        best_trans_prob = prob
                        best_trans_tag = prev_tag

                path_probability[tag].append(best_trans_prob)
                backpointer[tag].append(best_trans_tag)

        # termination step
        best_path_prob = -2.2250738585072014e+308
        best_path_pointer = None
        for tag in init_dist:
            if path_probability[tag][-1] > best_path_prob:
                best_path_prob = path_probability[tag][-1]
                best_path_pointer = tag

        predictions = [best_path_pointer]

        # make the predictions path
        for i in reversed(range(1, T)):
            predictions.append(backpointer[predictions[-1]][i])

        predictions.reverse()

        print(f'sentence:       {sentence}')
        print(f'hidden s:       {hidden_state}')
        print(f'predictions:    {predictions}')

        is_correct = create_confusion_matrix(predictions, hidden_state, confusion_matrices, is_correct)

    print(f'\nmodel got {is_correct} samples correct out of {test_size}')
    print(f'accuracy: {is_correct / test_size}')

    error_rate_hmm = (test_size - is_correct) / test_size
    most_frequent_tag = max(_tag_freq, key=_tag_freq.get)
    baseline_predictions = [most_frequent_tag] * len(testing_list)

    baseline_is_correct = sum(1 for i in range(len(testing_list)) if baseline_predictions[i] == testing_list[i][1])
    error_rate_baseline = (test_size - baseline_is_correct) / test_size

    print(f"Error rate for the HMM model: {error_rate_hmm}")
    print(f"Error rate for the most frequent tag baseline: {error_rate_baseline}")



In [31]:
CORPUS = brown.tagged_words(categories='news', tagset='universal')
CORPUS_SIZE = len(brown.tagged_words(categories='news'))

CUT_OFF = math.floor(CORPUS_SIZE * 0.75)

# section off training and testing lists from corpus
training_list = CORPUS[:CUT_OFF]
testing_list = CORPUS[CUT_OFF:]

tag_frequency, word_per_tag_frequency, tag_bigram, init_dist = collect_probabilities(training_list)
viterbi(testing_list, tag_frequency, word_per_tag_frequency, tag_bigram, init_dist)


Streaming output truncated to the last 5000 lines.
predictions:    ['DET', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'ADV', 'ADP', 'NOUN', '.']
DET: {'TP': 2300, 'FP': 185, 'TN': 16635, 'FN': 24}
NOUN: {'TP': 4786, 'FP': 1152, 'TN': 13057, 'FN': 149}
ADJ: {'TP': 963, 'FP': 89, 'TN': 17494, 'FN': 598}
VERB: {'TP': 2198, 'FP': 101, 'TN': 16308, 'FN': 537}
ADP: {'TP': 2354, 'FP': 479, 'TN': 16285, 'FN': 26}
.: {'TP': 2333, 'FP': 13, 'TN': 16798, 'FN': 0}
ADV: {'TP': 595, 'FP': 17, 'TN': 18281, 'FN': 251}
CONJ: {'TP': 564, 'FP': 2, 'TN': 18562, 'FN': 16}
PRT: {'TP': 222, 'FP': 8, 'TN': 18639, 'FN': 275}
PRON: {'TP': 510, 'FP': 2, 'TN': 18599, 'FN': 33}
NUM: {'TP': 267, 'FP': 0, 'TN': 18769, 'FN': 108}
X: {'TP': 4, 'FP': 0, 'TN': 19109, 'FN': 31}

Tag with the most false positives is: NOUN with 1152 counts.
Tag with the most false negative is:  ADJ with 598 counts.
confusion_matrices: {'DET': {'TP': 2300, 'FP': 185, 'TN': 16635, 'FN': 24}, 'NOUN': {'TP': 4786, 'FP': 1152, 'TN': 

# **Analyze**

First, let's look at the confusion matrix for each tag. The confusion matrix shows the number of true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN) for each tag.

The tag with the most false positives is NOUN, with 1469 false positives. This means that the model often mistakes other tags for NOUN.
The tag with the most false negatives is ADJ, with 735 false negatives. This means that the model often fails to tag words as ADJ when it should.
Next, let's look at the sentence and the predicted tags. We can see that the model made some mistakes in this sentence. For example, the word "one" is tagged as NOUN instead of NUM, and the word "reasons" is tagged as NOUN instead of VERB. These mistakes may be due to the fact that the model is based on probabilities and is not always accurate.

Overall, the model seems to perform well for some tags, such as DET, VERB, ADP, ., ADV, CONJ, PRT, PRON, and NUM, which have high true positive rates and low false positive and false negative rates. However, the model struggles with NOUN and ADJ, which have high false positive and false negative rates. This may be due to the fact that NOUN and ADJ are more ambiguous tags that can be confused with other tags.

To improve the performance of the model, we could consider using a more complex model, such as a conditional random field (CRF) or a recurrent neural network (RNN), which can capture more complex dependencies between tags. We could also consider using additional features, such as word shape or context, to improve the accuracy of the model.

# **For Last Output:**

In [33]:
def calculate_metrics(confusion_matrix):
    metrics = {}
    for tag, values in confusion_matrix.items():
        tp = values['TP']
        fp = values['FP']
        tn = values['TN']
        fn = values['FN']

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics[tag] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

    return metrics

In [34]:
confusion_matrices = {'DET': {'TP': 2970, 'FP': 242, 'TN': 21897, 'FN': 30}, 'NOUN': {'TP': 6395, 'FP': 1469, 'TN': 17076, 'FN': 199}, 'ADJ': {'TP': 1264, 'FP': 121, 'TN': 23019, 'FN': 735}, 'VERB': {'TP': 2909, 'FP': 128, 'TN': 21401, 'FN': 701}, 'ADP': {'TP': 3158, 'FP': 634, 'TN': 21314, 'FN': 33}, '.': {'TP': 3089, 'FP': 14, 'TN': 22036, 'FN': 0}, 'ADV': {'TP': 749, 'FP': 18, 'TN': 24043, 'FN': 329}, 'CONJ': {'TP': 715, 'FP': 3, 'TN': 24405, 'FN': 16}, 'PRT': {'TP': 291, 'FP': 12, 'TN': 24464, 'FN': 372}, 'PRON': {'TP': 607, 'FP': 2, 'TN': 24485, 'FN': 45}, 'NUM': {'TP': 344, 'FP': 1, 'TN': 24647, 'FN': 147}, 'X': {'TP': 4, 'FP': 0, 'TN': 25098, 'FN': 37}}

metrics = calculate_metrics(confusion_matrices)

sorted_tags = sorted(metrics.items(), key=lambda x: x[1]['f1_score'])

lowest_f1_tags = [tag[0] for tag in sorted_tags[:3]]
print(f"Tags with the lowest F1-scores (most frequent errors): {lowest_f1_tags}")


Tags with the lowest F1-scores (most frequent errors): ['X', 'PRT', 'ADJ']
